# Chapter 3 Storage

- [1. Set OpenAI API Key](#1. Set OpenAI-API-Key)
- [2. Dialogue Cache Storage](#2. Dialogue Cache Storage)
- [2.1 English Version](#2.1-English Version)
- [2.1.1 Initialize Dialogue Model](#2.1.1-Initialize Dialogue Model)
- [2.1.2 First Round Dialogue](#2.1.2-First Round Dialogue)
- [2.1.3 Second Round Dialogue](#2.1.3-Second Round Dialogue)
- [2.1.4 Third Round Dialogue](#2.1.4-Third Round Dialogue)
- [2.1.5 View Storage Cache](#2.1.5-View Storage Cache)
- [2.1.6 Add Content Directly to Storage Cache](#2.1.6-Add Content Directly to Storage Cache)
- [2.2 Chinese Version](#2.2-Chinese Version)
- [2.1.1 Initialize the dialogue model](#2.1.1- Initialize the dialogue model)
- [2.1.2 First round of dialogue](#2.1.2-First round of dialogue)
- [2.1.3 Second round of dialogue](#2.1.3-Second round of dialogue)
- [2.1.4 Third round of dialogue](#2.1.4-Third round of dialogue)
- [2.1.5 View storage cache](#2.1.5-View storage cache)
- [2.1.6 Add content directly to storage cache](#2.1.6-Add content directly to storage cache)
- [2.3 Summary](#2.3-Summary)
- [Three, dialogue cache window storage](#Three, dialogue cache window storage)
- [3.1 English version](#3.1-English version)
- [3.1.1 Add two rounds of dialogue to window storage](#3.1.1-Add two rounds of dialogue to window storage)
- [3.1.2 Apply window storage in dialogue chain](#3.1.2-Apply window storage in dialogue chain)
- [3.2 Chinese version](#3.2-Chinese version)
- [3.1.1 Add two rounds of conversations to window storage](#3.1.1-Add two rounds of conversations to window storage)
- [3.1.2 Apply window storage in conversation chain](#3.1.2-Apply window storage in conversation chain)
- [IV. Conversation token cache storage](#IV. Conversation token cache storage)
- [4.1 English version](#4.1-English version)
- [4.2 Chinese version](#4.2-Chinese version)
- [4.3 Supplement](#4.3-Supplement)
- [V. Conversation summary cache storage](#V. Conversation summary cache storage)
- [5.1 English version](#5.1-English version)
- [5.1.1 Use conversation summary cache storage](#5.1.1-Use conversation summary cache storage)
- [5.1.2 Conversation chain based on conversation summary cache storage](#5.1.2-Conversation chain based on conversation summary cache storage)
- [5.2 Chinese version](#5.2-Chinese version)
- [5.2.1 Use conversation summary cache storage](#5.2.1-Use conversation summary cache storage)
- [5.1.2 Conversation chain based on conversation summary cache storage](#5.1.2-Conversation chain based on conversation summary cache storage)

When you interact with those language models, they don't remember what you had communicated with them before, which is a big problem when we build some applications (such as chatbots) -- it doesn't seem smart enough! Therefore, in this section we will introduce the storage module in LangChain, that is, how to embed previous conversations into the language model so that it has the ability of continuous conversation.

When using the storage (Memory) module in LangChain, it can help save and manage historical chat messages, as well as build knowledge about specific entities. These components can store information across multiple rounds of conversations and allow tracking of specific information and context during the conversation.

LangChain provides multiple storage types. Among them, buffer storage allows the most recent chat messages to be retained, and summary storage provides a summary of the entire conversation. Entity storage allows information about specific entities to be retained in multiple rounds of conversations. These memory components are modular and can be used in combination with other components to enhance the robot's conversation management capabilities. The storage module can be accessed and updated through simple API calls, allowing developers to more easily manage and maintain conversation history.

This course mainly introduces four of the storage modules, and other modules can be viewed in the document for learning.
- Conversation Buffer Memory
- Conversation Buffer Window MemoryonBufferWindowMemory）
- Conversation Token Buffer Memory
- Conversation Summary Buffer Memory

In LangChain, storage refers to the short-term memory of the large language model (LLM). Why is it short-term memory? That's because after the LLM is trained (some long-term memory is obtained), its parameters will not change due to user input. When the user talks to the trained LLM, the LLM will temporarily remember the user's input and the output it has generated in order to predict the subsequent output, and after the model output is completed, it will "forget" the previous user's input and its output. Therefore, this information can only be called the short-term memory of the LLM. 

In order to extend the retention time of the LLM short-term memory, it is necessary to use some external storage methods to memorize it, so that when the user talks to the LLM, the LLM can know as much as possible about the historical conversation information between the user and it.

## 1. Set OpenAI API Key

Log in to [OpenAI account](https://platform.openai.com/account/api-keys) to get API Key, and then set it as an environment variable.

- If you want to set it as a global environment variable, you can refer to [Zhihu article](https://zhuanlan.zhihu.com/p/627665725).

- If you want to set it as a local/project environment variable, create a `.env` file in this file directory, open the file and enter the following content.

<p style="font-family:verdana; font-size:12px;color:green">
OPENAI_API_KEY="your_api_key" 
</p>

Replace "your_api_key" with your own API Key

In [ ]:
# Download the required packages python-dotenv and openai
# If you need to view the installation process log, you can remove -q
!pip install -q python-dotenv
!pip install -q openai

In [1]:
import os

import openai
from dotenv import find_dotenv, load_dotenv

# Read local/project environment variables.

# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# Get the environment variable OPENAI_API_KEY
openai.api_key = os.environ["OPENAI_API_KEY"]

## 2. Conversation Cache Storage

### 2.1 English version

#### 2.1.1 Initialize the dialogue model

In [47]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [48]:
# Here we set the parameter temperature to 0.0 to reduce the randomness of the generated answers.
# If you want to get different and innovative answers every time, you can try increasing this parameter.
llm = ChatOpenAI(temperature=0.0)  

memory = ConversationBufferMemory()

# Create a new ConversationChain Class instance
# When the verbose parameter is set to True, the program will output more detailed information to provide more debugging or runtime information.
# On the contrary, when the verbose parameter is set to False, the program will run in a more concise way and only output key information.
conversation = ConversationChain(llm=llm, memory = memory, verbose=True )

#### 2.1.2 First round of dialogue

When we ran the prediction (<font style="font-family:verdana; font-size:13.5px;color:#0055AA">predict</font>), some prompts were generated, as you can see below, he said "The following is a friendly conversation between a human and an AI, the AI ​​is talkative" and so on, this is actually the prompt generated by LangChain to make the system have a hopeful and friendly conversation, and the conversation must be saved, and the prompt for the current completed model chain.

In [49]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

#### 2.1.3 Second round of dialogue

When we have the second round of conversation, it will keep the above prompts

In [50]:
conversation.predict(input="What is 1+1?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

#### 2.1.4 The third round of dialogue

In order to verify whether he remembered the previous conversation, we asked him to answer what he had said before (my name). We can see that he did output the correct name, so this conversation chain will become longer and longer as it goes on.

In [51]:
conversation.predict(input="What is my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

#### 2.1.5 View storage cache

Storage cache (<font style="font-family:verdana; font-size:13.5px">memory.</font><font style="font-family:verdana; font-size:13.5px;color:#0055AA">buffer</font>)
Stores all conversation information so far

In [52]:
print(memory.buffer) 

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.


It is also possible to print the history of messages in the cache with load_memory_variables({}) . The {} here is an empty dictionary, there are some more advanced features that allow the user to use more complex input, but we won't be discussing them in this short course, so don't worry about why there is an empty curly brace here.

In [53]:
print(memory.load_memory_variables({}))

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Andrew."}


#### 2.1.6 Add content directly to the storage cache

In [54]:
memory = ConversationBufferMemory()  # 新建一个空的对话缓存记忆

In [55]:
memory.save_context({"input": "Hi"}, {"output": "What's up"})  # 向缓存区添加指定对话的输入输出

In [56]:
print(memory.buffer)  # 查看缓存区结果

Human: Hi
AI: What's up


In [57]:
print(memory.load_memory_variables({}))# 再次加载记忆变量

{'history': "Human: Hi\nAI: What's up"}


Continue to add new content, and the conversation history will be saved!

In [58]:
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [59]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### 2.2 Chinese version

#### 2.1.1 Initialize the dialogue model

In [30]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [33]:
llm = ChatOpenAI(temperature=0.0)  

memory = ConversationBufferMemory()

conversation = ConversationChain(llm=llm, memory = memory, verbose=True )



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好, 我叫皮皮鲁
AI:

> Finished chain.


'你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？'

#### 2.1.2 First round of dialogue

In [ ]:
conversation.predict(input="你好, 我叫皮皮鲁")

#### 2.1.3 Second round of dialogue

In [34]:
conversation.predict(input="1+1等于多少？")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？
Human: 1+1等于多少？
AI:

> Finished chain.


'1+1等于2。'

#### 2.1.4 The third round of dialogue

In [ ]:
conversation.predict(input="我叫什么名字？")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？
Human: 1+1等于多少？
AI: 1+1等于2。
Human: What is my name?
AI: 你的名字是皮皮鲁。
Human: 我叫什么名字？
AI:

> Finished chain.


'你叫皮皮鲁。'

#### 2.1.5 View storage cache

In [37]:
print(memory.buffer) 

Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？
Human: 1+1等于多少？
AI: 1+1等于2。
Human: What is my name?
AI: 你的名字是皮皮鲁。
Human: 我叫什么名字？
AI: 你叫皮皮鲁。


In [38]:
print(memory.buffer)  

Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？
Human: 1+1等于多少？
AI: 1+1等于2。
Human: What is my name?
AI: 你的名字是皮皮鲁。
Human: 我叫什么名字？
AI: 你叫皮皮鲁。


In [39]:
print(memory.load_memory_variables({}))

{'history': 'Human: 你好, 我叫皮皮鲁\nAI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？\nHuman: 1+1等于多少？\nAI: 1+1等于2。\nHuman: What is my name?\nAI: 你的名字是皮皮鲁。\nHuman: 我叫什么名字？\nAI: 你叫皮皮鲁。'}


In [40]:
memory.load_memory_variables({})

{'history': 'Human: 你好, 我叫皮皮鲁\nAI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？\nHuman: 1+1等于多少？\nAI: 1+1等于2。\nHuman: What is my name?\nAI: 你的名字是皮皮鲁。\nHuman: 我叫什么名字？\nAI: 你叫皮皮鲁。'}

#### 2.1.6 Add content directly to the storage cache

In [42]:
memory = ConversationBufferMemory()
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西'}

In [43]:
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [44]:
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: Not much, just hanging\nAI: Cool'}

In [45]:
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: Not much, just hanging\nAI: Cool\nHuman: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

### 2.3 Summary

When we use a large language model for a chat conversation, the large language model itself is actually stateless. The language model itself does not remember the historical conversation so far. Each call to the API node is independent. The memory can store all the terms or conversations so far and input or attach context to the LLM to generate output. It looks like it remembers what was said before when it is in the next round of conversation.

## 3. Conversation Cache Window Storage

As conversations become longer, the amount of memory required becomes very long. The cost of sending a large number of tokens to LLM will also become more expensive, which is why the API call fee is usually charged based on the number of tokens it needs to process.

In response to the above problems, LangChain also provides several convenient storage methods to save historical conversations. Among them, the conversation cache window storage only retains a window size of conversations. It only uses the most recent n interactions. This can be used to keep a sliding window of recent interactions so that the buffer is not too large

### 3.1 English version

#### 3.1.1 Add two rounds of dialogue to window storage

In [65]:
from langchain.memory import ConversationBufferWindowMemory

# k is the window parameter, k=1 means only one conversation memory is kept
memory = ConversationBufferWindowMemory(k=1)  

In [66]:
# Add two rounds of dialogue to memory
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [67]:
# And view the current record of the memory variable
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

#### 3.1.2 Using window storage in a dialog chain

In [68]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False  )

Note here! Since a window of memory is used here, only one round of historical messages can be saved. Therefore, the AI ​​cannot know the name you mentioned in the first round of conversation. It can only remember the conversation information of the previous round (the second round) at most.

In [69]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [70]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [71]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information."

### 3.2 Chinese version

#### 3.1.1 Add two rounds of dialogue to window storage

In [72]:
from langchain.memory import ConversationBufferWindowMemory

# k=1 means only one conversation memory is kept
memory = ConversationBufferWindowMemory(k=1)  
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

{'history': 'Human: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

#### 3.1.2 Using window storage in a dialog chain

In [74]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False  )
print(conversation.predict(input="你好, 我叫皮皮鲁"))
print(conversation.predict(input="1+1等于多少？"))
print(conversation.predict(input="我叫什么名字？"))

你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？
1+1等于2。
很抱歉，我无法知道您的名字。


## 4. Conversation token cache storage

With conversation token cache memoization, the memory will limit the number of tokens saved. If the number of tokens exceeds the specified number, it will cut off the earlier part of the conversation to keep the number of tokens corresponding to the most recent communication, but not exceeding the token limit.

In [76]:
!pip install -q tiktoken            

In [77]:
from langchain.llms import OpenAI
from langchain.memory import ConversationTokenBufferMemory

### 4.1 English version
Added dialog to Token cache storage, limited the number of tokens, and tested

In [78]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

In [79]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

You can see that the tokens that exceeded the limit have been discarded!!!

### 4.2 Chinese version

In [80]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "朝辞白帝彩云间，"}, {"output": "千里江陵一日还。"})
memory.save_context({"input": "两岸猿声啼不住，"}, {"output": "轻舟已过万重山。"})
memory.load_memory_variables({})

{'history': 'AI: 轻舟已过万重山。'}

### 4.3 Supplement

ChatGPT uses a Byte Pair Encoding (BPE)-based method for tokenization (splitting input text into tokens). BPE is a common tokenization technique that splits input text into smaller subword units. 

OpenAI has released a new open source Python library [tiktoken](https://github.com/openai/tiktoken) on its official GitHub. This library is mainly used to calculate the number of tokens. Compared with HuggingFace's tokenizer, its speed has increased several times. 

For specific token calculation methods, especially the difference between Chinese characters and English words, please refer to [Zhihu article](https://www.zhihu.com/question/594159910) for specific lessons.

## 5. Conversation summary cache storage

Conversation summary cache storage, **use LLM to write a summary of the historical conversation so far**, and save it

In [82]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

### 5.1 English version

#### 5.1.1 Using the conversation summary cache

Create a long string containing someone's schedule

In [84]:
# Create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

# Using conversation summary cache memory
llm = ChatOpenAI(temperature=0.0)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100) 
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context(
    {"input": "What is on the schedule today?"}, {"output": f"{schedule}"}
)

In [89]:
print(memory.load_memory_variables({})['history'])

System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.


#### 5.1.2 Conversation chain based on conversation summary cache storage
Based on the above conversation summary cache storage, create a new conversation chain

In [90]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [91]:
conversation.predict(input="What would be a good demo to show?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'

In [92]:
print(memory.load_memory_variables({})['history'])

System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.


### 5.2 Chinese version

#### 5.2.1 Using the conversation summary cache

Create a long string containing someone's schedule

In [97]:
# Create a long string
schedule = "在八点你和你的产品团队有一个会议。 \
你需要做一个PPT。 \
上午9点到12点你需要忙于LangChain。\
Langchain是一个有用的工具，因此你的项目进展的非常快。\
中午，在意大利餐厅与一位开车来的顾客共进午餐 \
走了一个多小时的路程与你见面，只为了解最新的 AI。 \
确保你带了笔记本电脑可以展示最新的 LLM 样例."

llm = ChatOpenAI(temperature=0.0)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.save_context({"input": "今天的日程安排是什么？"}, {"output": f"{schedule}"})

In [98]:
print(memory.load_memory_variables({})['history'])

System: The human introduces themselves as Pipilu and the AI introduces themselves as Luxixi. They express happiness at becoming friends and decide to go on an adventure together. The human asks about the schedule for the day. The AI informs them that they have a meeting with their product team at 8 o'clock and need to prepare a PowerPoint presentation. From 9 am to 12 pm, they will be busy with LangChain, a useful tool that helps their project progress quickly. At noon, they will have lunch with a customer who has driven for over an hour just to learn about the latest AI. The AI advises the human to bring their laptop to showcase the latest LLM samples.


#### 5.1.2 Conversation chain based on conversation summary cache storage
Based on the above conversation summary cache storage, create a new conversation chain

In [99]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [100]:
conversation.predict(input="展示什么样的样例最好呢？")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces themselves as Pipilu and the AI introduces themselves as Luxixi. They express happiness at becoming friends and decide to go on an adventure together. The human asks about the schedule for the day. The AI informs them that they have a meeting with their product team at 8 o'clock and need to prepare a PowerPoint presentation. From 9 am to 12 pm, they will be busy with LangChain, a useful tool that helps their project progress quickly. At noon, they will have lunch with a customer who has driven for over an hour just to learn about the latest AI. The AI advises the human to bring their laptop to showcase the latest LLM samples.
Human: 展示什么样的样例最好呢？
AI:

> Fini

'展示一些具有多样性和创新性的样例可能是最好的选择。你可以选择展示一些基于图像识别的样例，比如人脸识别、物体识别等。另外，你也可以展示一些自然语言处理方面的样例，比如文本生成、情感分析等。最重要的是选择那些能够展示出你们团队的技术实力和创造力的样例。'

In [101]:
memory.load_memory_variables({})  # 摘要记录更新了

{'history': "System: The human introduces themselves as Pipilu and the AI introduces themselves as Luxixi. They express happiness at becoming friends and decide to go on an adventure together. The human asks about the schedule for the day. The AI informs them that they have a meeting with their product team at 8 o'clock and need to prepare a PowerPoint presentation. From 9 am to 12 pm, they will be busy with LangChain, a useful tool that helps their project progress quickly. At noon, they will have lunch with a customer who has driven for over an hour just to learn about the latest AI. The AI advises the human to bring their laptop to showcase the latest LLM samples. The human asks what kind of samples would be best to showcase. The AI suggests that showcasing diverse and innovative samples would be the best choice. They recommend selecting samples based on image recognition, such as face recognition and object recognition. Additionally, they suggest showcasing samples related to natur